In [ ]:
# ===============================================
# GRASP Solver for Product Line Optimization
# ===============================================
# Author: Filipa Mota
# Date: 2025-11-11
#
# Reads all unsolved instances (e.g. 12-18-2.txt),
# solves each with GRASP metaheuristic, and saves
# results to /Unsolved/GRASP_Solutions.csv
# ===============================================

import os
import time
import numpy as np
import pandas as pd
from copy import deepcopy

# ------------------------------------------------
# 1️⃣ Instance Parser — identical structure as MIP
# ------------------------------------------------
def parse_instance(path):
    """
    Reads a .txt instance file such as 12-8-7.txt
    Returns dict with all relevant data arrays
    """
    with open(path, "r") as f:
        lines = [l.strip() for l in f if l.strip()]

    n = int(lines[0])   # products
    m = int(lines[1])   # customer segments
    f_classes = int(lines[2])  # manufacturing families

    # Extract numeric data (simplified, adapt if needed)
    # We'll just collect all floats, then reshape later
    floats = []
    for l in lines[4:]:
        try:
            floats.extend([float(x) for x in l.split()])
        except:
            pass

    data = {
        "n_products": n,
        "n_segments": m,
        "n_families": f_classes,
        # You can add more specific parsing below
    }
    return data


# ------------------------------------------------
# 2️⃣ Helper: Profit evaluation function
# ------------------------------------------------
def profit_function(selected_products, utilities, prices, costs, setup_class, setup_prod, family_map):
    """
    Computes total profit for a candidate solution.
    Replace this with your exact formulation.
    """
    # Placeholder profit: sum of (price - cost)
    # minus setup costs proportional to selected classes
    y = np.zeros(len(prices))
    y[selected_products] = 1
    profit = np.sum(y * (prices - costs))
    active_fams = set(family_map[j] for j in selected_products)
    profit -= sum(setup_class[k] for k in active_fams)
    profit -= sum(setup_prod[j] for j in selected_products)
    return profit
# ------------------------------------------------
# 3️⃣ Construction phase
# ------------------------------------------------
def constructive_phase(data, prices, costs, setup_class, setup_prod, family_map, rcl_size=3):
    """
    Greedy randomized construction of feasible solution
    """
    n = data["n_products"]
    current = []
    available = list(range(n))

    while available:
        # Evaluate marginal profit for each remaining product
        marginals = []
        for j in available:
            profit_if_add = profit_function(current + [j], prices, costs, setup_class, setup_prod, family_map)
            base_profit = profit_function(current, prices, costs, setup_class, setup_prod, family_map)
            marginals.append((j, profit_if_add - base_profit))

        # Restricted Candidate List (RCL)
        marginals.sort(key=lambda x: x[1], reverse=True)
        rcl = marginals[:max(1, rcl_size)]
        chosen = np.random.choice([j for j, _ in rcl])
        if marginals[0][1] <= 0:
            break
        current.append(chosen)
        available.remove(chosen)

    return list(set(current))


# ------------------------------------------------
# 4️⃣ Local search
# ------------------------------------------------
def local_search(solution, data, prices, costs, setup_class, setup_prod, family_map, max_iter=50):
    """
    Swap-based local search: try to improve profit by
    adding/removing/replacing products.
    """
    best = deepcopy(solution)
    best_profit = profit_function(best, prices, costs, setup_class, setup_prod, family_map)
    n = data["n_products"]
    for _ in range(max_iter):
        improved = False
        for j_out in best:
            for j_in in range(n):
                if j_in not in best:
                    new = [j for j in best if j != j_out] + [j_in]
                    new_profit = profit_function(new, prices, costs, setup_class, setup_prod, family_map)
                    if new_profit > best_profit:
                        best, best_profit = new, new_profit
                        improved = True
        if not improved:
            break
    return best, best_profit


# ------------------------------------------------
# 5️⃣ GRASP Metaheuristic Loop
# ------------------------------------------------
def grasp_solver(instance_path, max_iter=200, rcl_size=3):
    """
    Executes GRASP algorithm on one instance.
    """
    data = parse_instance(instance_path)
    n = data["n_products"]

    # Example random placeholder data — replace with parsed values
    np.random.seed(0)
    prices = np.random.uniform(10, 20, n)
    costs = np.random.uniform(5, 10, n)
    setup_class = np.random.uniform(100, 200, data["n_families"])
    setup_prod = np.random.uniform(20, 50, n)
    family_map = np.random.randint(0, data["n_families"], n)

    best_profit, best_sol = -np.inf, []
    start = time.time()

    for _ in range(max_iter):
        sol = constructive_phase(data, prices, costs, setup_class, setup_prod, family_map, rcl_size)
        sol, prof = local_search(sol, data, prices, costs, setup_class, setup_prod, family_map)
        if prof > best_profit:
            best_profit, best_sol = prof, sol

    runtime = time.time() - start
    name = os.path.basename(instance_path).replace(".txt", "")
    return {"instance": name, "profit": best_profit, "products": best_sol, "runtime": runtime}


# ------------------------------------------------
# 6️⃣ Batch runner for all /Unsolved/ files
# ------------------------------------------------
def solve_all_unsolved(folder="Unsolved", output_csv="Unsolved/GRASP_Solutions.csv"):
    results = []
    files = [f for f in os.listdir(folder) if f.endswith(".txt")]
    for f in files:
        path = os.path.join(folder, f)
        print(f"Solving {f} ...")
        res = grasp_solver(path)
        results.append(res)
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"✅ Results saved to {output_csv}")


# ------------------------------------------------
# 7️⃣ Run
# ------------------------------------------------
if __name__ == "__main__":
    solve_all_unsolved()


Solving 12-18-0.txt ...


TypeError: profit_function() missing 1 required positional argument: 'family_map'